In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
pd.set_option('display.max_rows', 8)

import collections

import warnings
# warnings.filterwarnings('ignore')
from matplotlib.backends.backend_pdf import PdfPages

from pathlib import Path

import db_queries as db
import vivarium_helpers.id_helper as idh
import gbd_mapping
from vivarium import Artifact

# Add the repo directory vivarium_research_ciff_sam/ to sys.path
import os, sys
repo_path = os.path.abspath('../..')
sys.path.append(repo_path)
# Assumes vivarium_research_ciff_sam/ is in sys.path
import model_validation.vivarium_transformed_output as vto
# import model_validation.vivarium_raw_output as vro
import model_validation.vivarium_output_processing as vop
import model_validation.ciff_sam_results as csr

!pwd
!whoami
!date

/ihme/homes/ndbs/vivarium_research_ciff_sam/model_validation/model2
ndbs
Tue Aug 31 14:42:29 PDT 2021


In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
data25 = csr.VivariumMeasures.cleaned_from_model_spec(2.5)
data25.table_names()

['wasting_transition_count',
 'wasting_state_person_time',
 'deaths',
 'population',
 'ylls',
 'ylds',
 'disease_state_person_time',
 'disease_transition_count']

In [4]:
data25.compute_person_time()
data25.table_names()

['wasting_transition_count',
 'wasting_state_person_time',
 'deaths',
 'population',
 'ylls',
 'ylds',
 'disease_state_person_time',
 'disease_transition_count',
 'person_time']

In [5]:
data25.wasting_state_person_time

,sex,year,wasting_state,measure,input_draw,scenario,value,age
0,female,2022,mild_child_wasting,state_person_time,29,baseline,414.090349,early_neonatal
1,female,2022,mild_child_wasting,state_person_time,29,baseline,1234.535250,late_neonatal
2,female,2022,mild_child_wasting,state_person_time,29,baseline,8926.954141,1-5_months
3,female,2022,mild_child_wasting,state_person_time,29,baseline,10434.387406,6-11_months
...,...,...,...,...,...,...,...,...
2876,male,2026,susceptible_to_child_wasting,state_person_time,946,baseline,30258.041068,1-5_months
2877,male,2026,susceptible_to_child_wasting,state_person_time,946,baseline,35695.241615,6-11_months
2878,male,2026,susceptible_to_child_wasting,state_person_time,946,baseline,70879.949350,12_to_23_months
2879,male,2026,susceptible_to_child_wasting,state_person_time,946,baseline,215444.187543,2_to_4


In [6]:
data25.person_time

,age,input_draw,measure,scenario,sex,year,value
0,1-5_months,29,person_time,baseline,female,2022,44140.585900
1,1-5_months,29,person_time,baseline,female,2023,43843.549624
2,1-5_months,29,person_time,baseline,female,2024,43946.220397
3,1-5_months,29,person_time,baseline,female,2025,43707.561944
...,...,...,...,...,...,...,...
836,all_ages,946,person_time,baseline,male,2023,521780.900753
837,all_ages,946,person_time,baseline,male,2024,529583.511294
838,all_ages,946,person_time,baseline,male,2025,532527.890486
839,all_ages,946,person_time,baseline,male,2026,532910.529774


# Compute SAM duration

= (person-time in SAM) / (# transitions into SAM)

In [7]:
data25.compute_sam_duration()

In [8]:
data25.sam_duration

,year,sex,age,input_draw,scenario,value,numerator_measure,denominator_measure,multiplier
0,2022,female,1-5_months,29,baseline,0.143876,state_person_time,transition_count,1
1,2022,female,1-5_months,223,baseline,0.146593,state_person_time,transition_count,1
2,2022,female,1-5_months,232,baseline,0.146573,state_person_time,transition_count,1
3,2022,female,1-5_months,357,baseline,0.135437,state_person_time,transition_count,1
...,...,...,...,...,...,...,...,...,...
716,2026,male,late_neonatal,650,baseline,0.172432,state_person_time,transition_count,1
717,2026,male,late_neonatal,680,baseline,0.150054,state_person_time,transition_count,1
718,2026,male,late_neonatal,829,baseline,0.159562,state_person_time,transition_count,1
719,2026,male,late_neonatal,946,baseline,0.151926,state_person_time,transition_count,1


In [9]:
vop.describe(365 * vop.value(data25.sam_duration))

count  \
age           denominator_measure multiplier numerator_measure scenario sex    year          
1-5_months    transition_count    1          state_person_time baseline female 2022   12.0   
                                                                               2023   12.0   
                                                                               2024   12.0   
                                                                               2025   12.0   
...                                                                                    ...   
late_neonatal transition_count    1          state_person_time baseline male   2023   12.0   
                                                                               2024   12.0   
                                                                               2025   12.0   
                                                                               2026   12.0   

                                                                                          mean  \
age           denominator_measure multiplier numerator_measure scenario sex    year              
1-5_months    transition_count    1          state_person_time baseline female 2022  54.080742   
                                                                               2023  53.633646   
                                                                               2024  53.708573   
                                                                               2025  53.739851   
...                                                                                        ...   
late_neonatal transition_count    1          state_person_time baseline male   2023  56.236192   
                                                                               2024  55.631115   
                                                                               2025  56.742480   
                                                                               2026  55.820676   

                                                                                          std  \
age           denominator_measure multiplier numerator_measure scenario sex    year             
1-5_months    transition_count    1          state_person_time baseline female 2022  2.206403   
                                                                               2023  1.916738   
                                                                               2024  1.889464   
                                                                               2025  1.796210   
...                                                                                       ...   
late_neonatal transition_count    1          state_person_time baseline male   2023  3.464888   
                                                                               2024  3.585755   
                                                                               2025  4.554065   
                                                                               2026  4.092007   

                                                                                           min  \
age           denominator_measure multiplier numerator_measure scenario sex    year              
1-5_months    transition_count    1          state_person_time baseline female 2022  49.434549   
                                                                               2023  50.078640   
                                                                               2024  50.175546   
                                                                               2025  50.271486   
...                                                                                        ...   
late_neonatal transition_count    1          state_person_time baseline male   2023  50.241827   
                                                                               2024  50.608843   
                     

In [10]:
1/vop.value(data25.sam_duration)

value
age           denominator_measure input_draw multiplier numerator_measure scenario sex    year          
1-5_months    transition_count    29         1          state_person_time baseline female 2022  6.950410
                                  223        1          state_person_time baseline female 2022  6.821591
                                  232        1          state_person_time baseline female 2022  6.822543
                                  357        1          state_person_time baseline female 2022  7.383500
...                                                                                                  ...
late_neonatal transition_count    650        1          state_person_time baseline male   2026  5.799380
                                  680        1          state_person_time baseline male   2026  6.664285
                                  829        1          state_person_time baseline male   2026  6.267150
                                  946        1          state_person_time baseline male   2026  6.582144

[720 rows x 1 columns]

In [11]:
data25.compute_sam_duration(strata='age')

In [12]:
vop.describe(1/vop.value(data25.sam_duration))

,,,,,count,mean,std,min,2.5%,50%,97.5%,max
age,denominator_measure,multiplier,numerator_measure,scenario,,,,,,,,
1-5_months,transition_count,1,state_person_time,baseline,12.0,6.815054,0.157524,6.596113,6.603078,6.795322,7.072381,7.081161
12_to_23_months,transition_count,1,state_person_time,baseline,12.0,6.464798,0.113226,6.200822,6.253584,6.461538,6.643023,6.680300
2_to_4,transition_count,1,state_person_time,baseline,12.0,6.594312,0.038105,6.528324,6.528440,6.608480,6.637580,6.641396
6-11_months,transition_count,1,state_person_time,baseline,12.0,6.698514,0.171382,6.338287,6.408336,6.696920,6.987305,7.049691
early_neonatal,transition_count,1,state_person_time,baseline,12.0,6.238405,0.073337,6.140244,6.143588,6.236663,6.353103,6.358320
late_neonatal,transition_count,1,state_person_time,baseline,12.0,6.490529,0.340494,5.808701,5.911979,6.447626,7.068608,7.129996


In [13]:
data25.compute_mam_duration()

In [15]:
365 * vop.value(data25.mam_duration)

value
age           denominator_measure input_draw multiplier numerator_measure scenario sex    year           
1-5_months    transition_count    29         1          state_person_time baseline female 2022  40.245868
                                  223        1          state_person_time baseline female 2022  38.551617
                                  232        1          state_person_time baseline female 2022  39.834246
                                  357        1          state_person_time baseline female 2022  38.772322
...                                                                                                   ...
late_neonatal transition_count    650        1          state_person_time baseline male   2026  38.286497
                                  680        1          state_person_time baseline male   2026  35.685426
                                  829        1          state_person_time baseline male   2026  36.438898
                                  946        1          state_person_time baseline male   2026  36.422863

[720 rows x 1 columns]

# Check MAM duration

person time in MAM/#transition into MAM = 1/(r3_ux + t2_mam + i_sam + death_rate)

* r3_ux = 0.512/63days
* t2_mam = 0.488/41.3days
* i_sam = transition counts to sam/pt in mam
* death rate = MAM death rate for the age/sex

In [17]:
data25.compute_mam_duration('age')
vop.describe(1/vop.value(data25.mam_duration))

,,,,,count,mean,std,min,2.5%,50%,97.5%,max
age,denominator_measure,multiplier,numerator_measure,scenario,,,,,,,,
1-5_months,transition_count,1,state_person_time,baseline,12.0,9.642899,0.137728,9.446445,9.447015,9.650405,9.831969,9.846348
12_to_23_months,transition_count,1,state_person_time,baseline,12.0,8.948815,0.097860,8.770894,8.778911,8.965023,9.081001,9.099300
2_to_4,transition_count,1,state_person_time,baseline,12.0,8.469425,0.053281,8.391200,8.396504,8.480601,8.556465,8.562436
6-11_months,transition_count,1,state_person_time,baseline,12.0,9.516540,0.137133,9.203036,9.263096,9.528334,9.743906,9.786546
early_neonatal,transition_count,1,state_person_time,baseline,12.0,8.984767,0.118001,8.786298,8.808522,8.965226,9.198165,9.209819
late_neonatal,transition_count,1,state_person_time,baseline,12.0,9.570323,0.211558,9.257089,9.269857,9.573808,9.866351,9.874116


In [22]:
pd.read_csv("../../wasting_transitions/cat_strat_mort_2021_07_13.csv")

,sex_id,age_group_id,cat,mortality_hazard,time_unit
0,1,4,cat1,0.052008,year
1,1,4,cat2,0.016679,year
2,1,4,cat3,0.007460,year
3,1,4,cat4,0.004459,year
...,...,...,...,...,...
12,2,5,cat1,0.014684,year
13,2,5,cat2,0.004728,year
14,2,5,cat3,0.002089,year
15,2,5,cat4,0.001258,year


In [19]:
repo_path

'/ihme/homes/ndbs/vivarium_research_ciff_sam'

In [21]:
pd.read_csv(f"{repo_path}/wasting_transitions/cat_strat_mort_2021_07_13.csv")

,sex_id,age_group_id,cat,mortality_hazard,time_unit
0,1,4,cat1,0.052008,year
1,1,4,cat2,0.016679,year
2,1,4,cat3,0.007460,year
3,1,4,cat4,0.004459,year
...,...,...,...,...,...
12,2,5,cat1,0.014684,year
13,2,5,cat2,0.004728,year
14,2,5,cat3,0.002089,year
15,2,5,cat4,0.001258,year


In [23]:
idh.ids_to_names('age_group', 4,5)

age_group_id
4    Post Neonatal
5           1 to 4
Name: age_group_name, dtype: object